In [1]:
import re, json, os, selenium, jsonpath, time, pandas as pd, urllib, requests
from bs4 import BeautifulSoup
from lib2to3.pgen2 import driver
# from macpath import join
from operator import contains
from turtle import title
from webbrowser import get
from selenium import webdriver
from selenium.webdriver import ChromeOptions
import time
import re
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
import random
from PIL import Image
import cv2
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By

In [3]:
keyword_df = pd.read_csv('./json_Art_list/df_total_ich_art_Final.csv')
keyword_df

,id,num,auto_id,order,title,type,num_first,num_sec,cate,project_num,rx_time,address,province,city,area,content,playtype,protect_unit
0,12408,Ⅱ-1,32,NaN,左权开花调,传统音乐,2,1,新增项目,Ⅱ-001_01_140722,"2006,(第一批)",140000.0,山西省左权县,140700.0,140722.0,左权县位于太行山主脉西侧、山西省东部边缘，古称辽州、辽县，后因纪念牺牲在此的抗日名将而改...,local,左权县民歌研究中心
1,12409,Ⅱ-2,33,NaN,河曲民歌,传统音乐,2,2,新增项目,Ⅱ-002_01_140930,"2006,(第一批)",140000.0,山西省河曲县,140900.0,140930.0,河曲位于山西省西北部，东界偏关、五寨，南界岢岚、保德，西、北隔黄河分别与陕西、内蒙相邻，...,local,河曲县二人台艺术研究中心
2,12410,Ⅱ-3,34,NaN,蒙古族长调民歌,传统音乐,2,3,新增项目,Ⅱ-003_01_150000,"2006,(第一批)",150000.0,内蒙古自治区,NaN,NaN,长调是蒙古语“乌日汀哆”的意译。“乌日汀”为“长久”、“永恒”之意，“哆”为“歌”之意。...,local,内蒙古自治区艺术研究所（内蒙古自治区艺术档案馆）
3,12411,Ⅱ-3,34,NaN,蒙古族长调民歌,传统音乐,2,3,扩展项目,Ⅱ-003_02_652800,"2008,(第二批)",650000.0,新疆维吾尔自治区巴音郭楞蒙古自治州,652800.0,NaN,长调系蒙古语“乌日汀哆”的意译，它是北方草原游牧民族在畜牧业生产劳动中创造的一种民歌，在...,local,新疆巴音郭楞蒙古自治州文化馆
4,12412,Ⅱ-3,34,NaN,蒙古族长调民歌,传统音乐,2,3,扩展项目,Ⅱ-003_02_654226,"2008,(第二批)",650000.0,新疆维吾尔自治区和布克赛尔蒙古自治县,654200.0,654226.0,长调系蒙古语“乌日汀哆”的意译，它是北方草原游牧民族在畜牧业生产劳动中创造的一种民歌，在...,local,和布克赛尔蒙古自治县非物质文化遗产保护中心
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,23550,Ⅶ-135,1483,NaN,瓷刻（大丰瓷刻）,传统美术,7,135,新增项目,NaN,"2021,(第五批)",320000.0,江苏省盐城市大丰区,320900.0,320982.0,None,local,盐城市大丰区瓷刻世家工艺厂
1886,23551,Ⅶ-136,1484,NaN,竹根雕（象山竹根雕）,传统美术,7,136,新增项目,NaN,"2021,(第五批)",330000.0,浙江省宁波市象山县,330200.0,330225.0,None,local,象山德和根艺美术馆
1887,23552,Ⅶ-137,1485,NaN,贝雕（北海贝雕）,传统美术,7,137,新增项目,NaN,"2021,(第五批)",450000.0,广西壮族自治区北海市,450500.0,NaN,None,local,北海市恒兴珠宝有限责任公司
1888,23553,Ⅶ-138,1486,NaN,骨角雕（合浦角雕）,传统美术,7,138,新增项目,NaN,"2021,(第五批)",450000.0,广西壮族自治区北海市合浦县,450500.0,450521.0,None,local,合浦金蝠角雕厂


In [4]:
keyword_df['title'] = keyword_df['title'].str.replace('（',' ')
keyword_df['title'] = keyword_df['title'].str.replace('）', '')
keyword_df['title'] = keyword_df['title'].str.replace(r'[^\u4e00-\u9fa5]',' ')
keyword_df['title'] = keyword_df['title'].str.strip()
keyword_df

C:\Users\Unknown\AppData\Local\Temp\ipykernel_128652\3365421286.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  keyword_df['title'] = keyword_df['title'].str.replace(r'[^\u4e00-\u9fa5]',' ')


,id,num,auto_id,order,title,type,num_first,num_sec,cate,project_num,rx_time,address,province,city,area,content,playtype,protect_unit
0,12408,Ⅱ-1,32,NaN,左权开花调,传统音乐,2,1,新增项目,Ⅱ-001_01_140722,"2006,(第一批)",140000.0,山西省左权县,140700.0,140722.0,左权县位于太行山主脉西侧、山西省东部边缘，古称辽州、辽县，后因纪念牺牲在此的抗日名将而改...,local,左权县民歌研究中心
1,12409,Ⅱ-2,33,NaN,河曲民歌,传统音乐,2,2,新增项目,Ⅱ-002_01_140930,"2006,(第一批)",140000.0,山西省河曲县,140900.0,140930.0,河曲位于山西省西北部，东界偏关、五寨，南界岢岚、保德，西、北隔黄河分别与陕西、内蒙相邻，...,local,河曲县二人台艺术研究中心
2,12410,Ⅱ-3,34,NaN,蒙古族长调民歌,传统音乐,2,3,新增项目,Ⅱ-003_01_150000,"2006,(第一批)",150000.0,内蒙古自治区,NaN,NaN,长调是蒙古语“乌日汀哆”的意译。“乌日汀”为“长久”、“永恒”之意，“哆”为“歌”之意。...,local,内蒙古自治区艺术研究所（内蒙古自治区艺术档案馆）
3,12411,Ⅱ-3,34,NaN,蒙古族长调民歌,传统音乐,2,3,扩展项目,Ⅱ-003_02_652800,"2008,(第二批)",650000.0,新疆维吾尔自治区巴音郭楞蒙古自治州,652800.0,NaN,长调系蒙古语“乌日汀哆”的意译，它是北方草原游牧民族在畜牧业生产劳动中创造的一种民歌，在...,local,新疆巴音郭楞蒙古自治州文化馆
4,12412,Ⅱ-3,34,NaN,蒙古族长调民歌,传统音乐,2,3,扩展项目,Ⅱ-003_02_654226,"2008,(第二批)",650000.0,新疆维吾尔自治区和布克赛尔蒙古自治县,654200.0,654226.0,长调系蒙古语“乌日汀哆”的意译，它是北方草原游牧民族在畜牧业生产劳动中创造的一种民歌，在...,local,和布克赛尔蒙古自治县非物质文化遗产保护中心
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,23550,Ⅶ-135,1483,NaN,瓷刻 大丰瓷刻,传统美术,7,135,新增项目,NaN,"2021,(第五批)",320000.0,江苏省盐城市大丰区,320900.0,320982.0,None,local,盐城市大丰区瓷刻世家工艺厂
1886,23551,Ⅶ-136,1484,NaN,竹根雕 象山竹根雕,传统美术,7,136,新增项目,NaN,"2021,(第五批)",330000.0,浙江省宁波市象山县,330200.0,330225.0,None,local,象山德和根艺美术馆
1887,23552,Ⅶ-137,1485,NaN,贝雕 北海贝雕,传统美术,7,137,新增项目,NaN,"2021,(第五批)",450000.0,广西壮族自治区北海市,450500.0,NaN,None,local,北海市恒兴珠宝有限责任公司
1888,23553,Ⅶ-138,1486,NaN,骨角雕 合浦角雕,传统美术,7,138,新增项目,NaN,"2021,(第五批)",450000.0,广西壮族自治区北海市合浦县,450500.0,450521.0,None,local,合浦金蝠角雕厂


In [5]:
keyword_df = keyword_df.drop_duplicates(subset=['title'], keep='first')
keyword_df = keyword_df.reset_index(drop=True)
keyword_df

,id,num,auto_id,order,title,type,num_first,num_sec,cate,project_num,rx_time,address,province,city,area,content,playtype,protect_unit
0,12408,Ⅱ-1,32,NaN,左权开花调,传统音乐,2,1,新增项目,Ⅱ-001_01_140722,"2006,(第一批)",140000.0,山西省左权县,140700.0,140722.0,左权县位于太行山主脉西侧、山西省东部边缘，古称辽州、辽县，后因纪念牺牲在此的抗日名将而改...,local,左权县民歌研究中心
1,12409,Ⅱ-2,33,NaN,河曲民歌,传统音乐,2,2,新增项目,Ⅱ-002_01_140930,"2006,(第一批)",140000.0,山西省河曲县,140900.0,140930.0,河曲位于山西省西北部，东界偏关、五寨，南界岢岚、保德，西、北隔黄河分别与陕西、内蒙相邻，...,local,河曲县二人台艺术研究中心
2,12410,Ⅱ-3,34,NaN,蒙古族长调民歌,传统音乐,2,3,新增项目,Ⅱ-003_01_150000,"2006,(第一批)",150000.0,内蒙古自治区,NaN,NaN,长调是蒙古语“乌日汀哆”的意译。“乌日汀”为“长久”、“永恒”之意，“哆”为“歌”之意。...,local,内蒙古自治区艺术研究所（内蒙古自治区艺术档案馆）
3,12413,Ⅱ-3,34,NaN,蒙古族长调民歌 巴尔虎长调,传统音乐,2,3,扩展项目,Ⅱ-003_04_150726,"2014,(第四批)",150000.0,内蒙古自治区新巴尔虎左旗,150700.0,150726.0,None,local,新巴尔虎左旗文化馆
4,23641,Ⅱ-3,34,NaN,蒙古族长调民歌 乌珠穆沁长调,传统音乐,2,3,扩展项目,NaN,"2021,(第五批)",150000.0,内蒙古自治区锡林郭勒盟西乌珠穆沁旗,152500.0,152526.0,None,local,西乌珠穆沁旗文化馆
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1568,23550,Ⅶ-135,1483,NaN,瓷刻 大丰瓷刻,传统美术,7,135,新增项目,NaN,"2021,(第五批)",320000.0,江苏省盐城市大丰区,320900.0,320982.0,None,local,盐城市大丰区瓷刻世家工艺厂
1569,23551,Ⅶ-136,1484,NaN,竹根雕 象山竹根雕,传统美术,7,136,新增项目,NaN,"2021,(第五批)",330000.0,浙江省宁波市象山县,330200.0,330225.0,None,local,象山德和根艺美术馆
1570,23552,Ⅶ-137,1485,NaN,贝雕 北海贝雕,传统美术,7,137,新增项目,NaN,"2021,(第五批)",450000.0,广西壮族自治区北海市,450500.0,NaN,None,local,北海市恒兴珠宝有限责任公司
1571,23553,Ⅶ-138,1486,NaN,骨角雕 合浦角雕,传统美术,7,138,新增项目,NaN,"2021,(第五批)",450000.0,广西壮族自治区北海市合浦县,450500.0,450521.0,None,local,合浦金蝠角雕厂


In [5]:
keyword_df.iloc[1008]

id                        13574
num                       Ⅳ-157
auto_id                    1119
order                       NaN
title                   阳戏 上河阳戏
type                       传统戏剧
num_first                     4
num_sec                     157
cate                       扩展项目
project_num     Ⅳ-157_04_431202
rx_time              2014,(第四批)
address                430000.0
province              湖南省怀化市鹤城区
city                   431200.0
area                   431202.0
content                    None
playtype                  local
protect_unit     怀化市鹤城区阳戏保护传承中心
Name: 1008, dtype: object

In [10]:
keyword_df[keyword_df['title'] == '舞']

,id,num,auto_id,order,title,type,num_first,num_sec,cate,project_num,rx_time,address,province,city,area,content,playtype,protect_unit
534,12978,Ⅲ-34,137,NaN,舞,传统舞蹈,3,34,新增项目,Ⅲ-034_01_513225,"2006,(第一批)",510000.0,四川省九寨沟县,513200.0,513225.0,㑇，白马人之方言，意为吉祥面具舞，汉语俗称“十二相舞”。它源于白马人崇尚“万物有灵”的原...,local,九寨沟县文化馆


In [6]:
keyword_df.iloc[410:412]

,id,num,auto_id,order,title,type,num_first,num_sec,cate,project_num,rx_time,address,province,city,area,content,playtype,protect_unit
410,12849,Ⅲ-4,107,16.0,龙舞 黄龙溪火龙灯舞,传统舞蹈,3,4,扩展项目,Ⅲ-004_02_510122,"2008,(第二批)",510000.0,四川省双流县,510100.0,510122.0,双流县 龙舞也称“舞龙”，民间又叫“耍龙”、“耍龙灯”、“舞龙灯”，是中国分布最广、影响最...,local,双流县文化馆
411,12850,Ⅲ-4,107,1.0,龙舞 浦东绕龙灯,传统舞蹈,3,4,扩展项目,Ⅲ-004_03_310115,"2011,(第三批)",310000.0,上海市浦东新区,310100.0,310115.0,“绕龙灯”是旧时上海浦东民众对舞龙活动的俗称，多出现在节日喜庆和求雨、禳灾、酬神、祈平安...,local,上海市浦东新区三林镇文化服务中心


In [6]:
keyword_dict = {}
for type_indf in keyword_df['type'].unique():
    keyword_dict[type_indf] = keyword_df[keyword_df['type'] == type_indf]['title'].tolist()
keyword_dict

{'传统音乐': ['左权开花调',
  '河曲民歌',
  '蒙古族长调民歌',
  '蒙古族长调民歌 巴尔虎长调',
  '蒙古族长调民歌 乌珠穆沁长调',
  '蒙古族呼麦',
  '当涂民歌',
  '巢湖民歌',
  '畲族民歌',
  '兴国山歌',
  '兴山民歌',
  '桑植民歌',
  '梅州客家山歌',
  '中山咸水歌',
  '崖州民歌',
  '儋州调声',
  '石柱土家啰儿调',
  '巴山背二歌',
  '傈僳族民歌',
  '紫阳民歌',
  '裕固族民歌',
  '花儿 莲花山花儿会',
  '花儿 松鸣岩花儿会',
  '花儿 二郎山花儿会',
  '花儿 老爷山花儿会',
  '花儿 丹麻土族花儿会',
  '花儿 瞿昙寺花儿会',
  '花儿 七里寺花儿会',
  '花儿 宁夏回族山花儿',
  '花儿 新疆花儿',
  '花儿 张家川花儿',
  '藏族拉伊',
  '藏族拉伊 那曲拉伊',
  '聊斋俚曲',
  '靖州苗族歌鼟',
  '川江号子',
  '南溪号子',
  '木洞山歌',
  '川北薅草锣鼓',
  '薅草锣鼓 武宁打鼓歌',
  '薅草锣鼓 宜昌薅草锣鼓',
  '薅草锣鼓 五峰土家族薅草锣鼓',
  '薅草锣鼓 兴山薅草锣鼓',
  '薅草锣鼓 宣恩薅草锣鼓',
  '薅草锣鼓 长阳山歌',
  '薅草锣鼓 川东土家族薅草锣鼓',
  '薅草锣鼓 金湖秧歌',
  '侗族大歌',
  '侗族琵琶歌',
  '哈尼族多声部民歌',
  '多声部民歌 潮尔道 蒙古族合声演唱',
  '多声部民歌 瑶族蝴蝶歌',
  '多声部民歌 壮族三声部民歌',
  '多声部民歌 羌族多声部民歌',
  '多声部民歌 硗碛多声部民歌',
  '多声部民歌 苗族多声部民歌',
  '多声部民歌 潮尔道 阿巴嘎潮尔',
  '多声部民歌 阿尔麦多声部民歌',
  '彝族海菜腔',
  '那坡壮族民歌',
  '澧水船工号子',
  '古琴艺术',
  '古琴艺术 虞山琴派',
  '古琴艺术 广陵琴派',
  '古琴艺术 金陵琴派',
  '古琴艺术 梅庵琴派',
  '古琴艺术 浙派',
  '古琴艺术 诸城派',
  '古琴艺术 岭南派',
  '蒙古族马头琴音乐',
  '蒙古族马头琴音乐 蒙古勒津马头琴音乐'

In [ ]:
len(keyword_df['title'])

## Process the useless files

In [ ]:
for ich_type in keyword_df['type'].unique():
    print(ich_type)
    if not os.path.exists('./{}'.format(ich_type)):
        os.makedirs('./{}'.format(ich_type), exist_ok=True)

In [ ]:
import shutil 
path1='./video_id_json/' 
files = os.listdir(path1)
for file in files: 
    path2 = keyword_df[keyword_df['title'] == file[:-5]]['type'].values[0]
    os.makedirs('./{}/{}'.format(path2, file[:-5]), exist_ok=True)
    path2 = './{}/{}/'.format(path2, file[:-5])
    shutil.move('./{}/{}'.format(path1, file), path2)

In [ ]:

path = './video_id_json/'
files = os.listdir(path)
print(len(files))
useful_json = []
for file in files:
        filepath = path + '/' + file
        if os.path.isfile(filepath):
            os.rename(filepath, filepath+'.json')
            # with open(filepath, 'r', encoding='UTF-8') as js_file:
            #     js = json.load(js_file)
            #     for key, value in js.items():
            #         if len(value)!=0:
            #             useful_json.append(file)
useful_json

In [ ]:
temp_list = [x[:-5] for x in useful_json]
for i in temp_list:
    if i not in keyword_df['title'].tolist():
        print(i)
remove_files = list(set(keyword_df['title'].tolist()).difference(set(temp_list)))
print(len(remove_files))
for rm_files in remove_files:
    os.remove(path+rm_files)

## Crawler Part


In [6]:
def setting_option():
    option = ChromeOptions()
    option.add_argument(
            'User-Agent: "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"'
        )
    option = ChromeOptions()
    option.add_argument(
            'user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"'
        )
    # option.add_argument('--proxy-server=http://182.106.220.252:9091')    
    option.add_experimental_option('excludeSwitches', ['enable-automation']) 
    option.add_experimental_option('useAutomationExtension', False)
    return option


In [7]:
def browser_initial(browser,keyword):
    url = 'https://www.douyin.com/search/' + keyword + '?publish_time=0&sort_type=0&source=switch_tab&type=video'
    browser.get(url)
    time.sleep(30) 
    browser.find_element_by_xpath('//*[@id="qdblhsHs"]/button').click()
    time.sleep(30)  
    get_cookies(browser)
    cookie_flag = 1

def get_cookies(browser):
    dictCookies = browser.get_cookies()
    jsonCookies = json.dumps(dictCookies)

    with open('./cookie_lib//DY_Cookies.json', 'w') as f:
        f.write(jsonCookies)

    browser.close()

In [8]:

# 获取滑块验证图片写入本地
def getCatpcha(verifyBgUrl, verifyPointUrl):
    # 下载图片资源
    # pic1 = requests.get(verifyBgUrl, proxies={"http": "http://{}".format(proxy)}).content
    # pic2 = requests.get(verifyPointUrl, proxies={"http": "http://{}".format(proxy)}).content
    pic1 = requests.get(verifyBgUrl).content
    pic2 = requests.get(verifyPointUrl).content
    # 增加保存图片的前缀
    fileFix = int(time.time())
    fileFix = str(fileFix)
    bg = fileFix + 'bg.jpg'
    point = fileFix + 'point.jpg'
    with open(bg, 'wb') as f:  # 要写入本地
        f.write(pic1)
    with open(point, 'wb') as f:  # 要写入本地
        f.write(pic2)
    return bg, point


# 获取滑块最终需要移动到到坐标X位置
def getTargetSliderPointX(verifyBgUrl, verifyPointUrl):
    # 下载图片并返回保存路径
    bgFilePath, pointFilePath = getCatpcha(verifyBgUrl, verifyPointUrl)
    bgImg = cv2.imread(bgFilePath)
    # 变成BRG格式来OpenCV处理
    pointImg = cv2.imread(pointFilePath)
    # 获取图像的边缘，Canny（图，阈值，阈值）
    bgEdge = cv2.Canny(bgImg, 100, 200)
    tpEdge = cv2.Canny(pointImg, 100, 200)
    bgPic = cv2.cvtColor(bgEdge, cv2.COLOR_GRAY2RGB)  # 颜色空间转换函数，cvtColor（图，要变成的格式）
    ptPic = cv2.cvtColor(tpEdge, cv2.COLOR_GRAY2RGB)
    # 缺口匹配
    res = cv2.matchTemplate(bgPic, ptPic, cv2.TM_CCOEFF_NORMED)
    # 寻找最优匹配
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    # 读取背景图片

    with Image.open(bgFilePath) as im:
            bgImgSize = im.size
    # 根据图片真实与表面大小的比例进行缩放
    x = max_loc[0] * bgImgSize[1] / bgImgSize[0]
    # 删除下载的文件
    os.unlink(bgFilePath)
    os.unlink(pointFilePath)
    return x

# 移动滑块 web=webdriver distance=水平移动距离x element=滑块xpath语法匹配
def startMove(web, distance, element):
    # element = web.find_element(By.XPATH, element)
    # 开始移动
#     js = "arguments[0].style = 'transform: translate({}px, 0px);".format(distance)
# # 执行js
#     web.execute_script(js, element)
    element = web.find_element(By.XPATH, element)
    ActionChains(web).click_and_hold(element).perform()
    time.sleep(0.5)
    while distance > 0:
        if distance > 20:
            # 如果距离大于20，就让他移动快一点
            span = random.randint(60, 70)
        else:
            # 快到缺口了，就移动慢一点
            span = random.randint(45, 50)
        ActionChains(web).move_by_offset(span, 0).perform()
        distance -= span
        time.sleep(random.randint(10, 50) / 100)

    ActionChains(web).move_by_offset(distance, 1).perform()
    ActionChains(web).release(on_element=element).perform()



def doMoveValidate(browser:webdriver):
    flag = False
    try:
        browser.implicitly_wait(5)
        verify_img = browser.find_element(By.XPATH, '//*[@id="captcha-verify-image"]') #读取要拼图的图片
        point_img = browser.find_element(By.XPATH,'//*[@id="captcha_container"]/div/div[2]/img[2]') #读取拼图的图片
        img_url = verify_img.get_attribute('src')
        img_url2 = point_img.get_attribute('src')
        print('成功获取图片url')
        startMove(web=browser, distance=getTargetSliderPointX(img_url,img_url2), element='//*[@id="secsdk-captcha-drag-wrapper"]/div[2]')

        time.sleep(5)
    except:
        flag = True
        print('Going on')
    return flag

In [12]:
browser = webdriver.Chrome(ChromeDriverManager().install(), options=setting_option())
browser.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {
            'source': 'Object.defineProperty(navigator, "webdriver", {get: () => undefined})'
        })
browser_initial(browser,'东方红')

## below is core part

In [9]:

def getVideoList_serach(keyword:str):
    start = time.time()
    flag = True
    # url = urllib.parse.quote(url)
    url = 'https://www.douyin.com/search/' + keyword + '?publish_time=0&sort_type=0&source=switch_tab&type=video'
    browser = webdriver.Chrome(ChromeDriverManager().install(), options=setting_option())
    browser.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {
            'source': 'Object.defineProperty(navigator, "webdriver", {get: () => undefined})'
        })
    browser.get(url)
    while True:
        end = time.time()
        if start - end > 140:
            flag = False
            break
        validate_f = doMoveValidate(browser)
        if validate_f:
            break
    # browser.maximize_window() 
    with open('./cookie_lib/DY_Cookies.json', 'r', encoding='utf-8') as f:
            listCookies = json.load(f)

    browser.delete_all_cookies()
    for cookie in listCookies: 
        if 'expiry' in cookie:
                del cookie["expiry"]
        browser.add_cookie(cookie)
    print('成功加载cookies')
    browser.refresh()
    browser.get(url)
    time.sleep(5)
    while True:
        end = time.time()
        if start - end > 260:
            flag = False
            break
        validate_f = doMoveValidate(browser)
        if validate_f:
            break
    # browser.maximize_window() 
    return browser, flag

def parseVideoList(browser:webdriver, flag:bool):
    if flag:
        video_id_lists = list(set(re.findall(r'<a href="//www.douyin.com/video/(.*?)"',browser.page_source)))
    else:
        video_id_lists = []
    browser.close()
    return video_id_lists

def getVideoInfo(video_id:str):
    url = 'https://www.douyin.com/video/' + video_id
    headers = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
    }

    with open('./DY_Cookies.json', 'r', encoding='utf-8') as f:
            listCookies = json.load(f)
    cookie_dict = {}
    for cookie in listCookies: 
        cookie_dict[cookie['name']] = cookie['value']

    # data = requests.get(url=url, headers=headers, cookies=cookie_dict, proxies={"http": "http://{}".format(proxy)})
    data = requests.get(url=url, headers=headers, cookies=cookie_dict)
    data.encoding = 'utf-8'
    return data.text

def parseVideoInfo(data:str):
    data_en = re.findall(r'<script id="RENDER_DATA" type="application/json">(.*?)</script>', data)[0]
    data_all = requests.utils.unquote(data_en)

    data_json = json.loads(data_all)

    video_url = 'https:' + jsonpath.jsonpath(data_json, '$..video.playAddr[1].src')[0]
    # print(video_url)

    comments = jsonpath.jsonpath(data_json, '$..comments[*]')
    # print(comments)

    video_metrics = jsonpath.jsonpath(data_json, '$..stats')[0]
    # print(video_metrics)

    title = jsonpath.jsonpath(data_json, '$..detail.desc')[0]
    # print(title)

    create_time = jsonpath.jsonpath(data_json, '$..detail.createTime')[0]
    # tag = list(filter(lambda x: x!='' , jsonpath.jsonpath(data_json, '$..detail.textExtra[*].hashtagName')))
    tag = jsonpath.jsonpath(data, '$..detail.textExtra[*].hashtagName')
    # print(tag)

    author_info_key = ['uid','nickname','followerCount','totalFavorited','enterpriseVerifyReason','customVerify']
    author_info_value = jsonpath.jsonpath(data_json, '$..detail.authorInfo[uid,nickname,followerCount,totalFavorited,enterpriseVerifyReason,customVerify]')
    author_info = dict(zip(author_info_key, author_info_value))
    # print(author_info)

    return {'video_url':video_url, 'comments':comments, 'video_metrics':video_metrics, 'create_time':create_time, 'title':title, 'tag':tag, 'author_info':author_info}

## multithread for selenium

In [ ]:
with open ('./first_machine_test.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
video_url = 'https:' + jsonpath.jsonpath(data, '$..video.playAddr[1].src')[0]
print(video_url)

comments = jsonpath.jsonpath(data, '$..comments[*]')
    # print(comments)

video_metrics = jsonpath.jsonpath(data, '$..stats')[0]
print(video_metrics)

title = jsonpath.jsonpath(data, '$..detail.desc')[0]
print(title)

create_time = jsonpath.jsonpath(data, '$..detail.createTime')[0]
print(create_time)
# tag = list(filter(lambda x: x!='' , jsonpath.jsonpath(data, '$..detail.textExtra[*].hashtagName')))
tag = jsonpath.jsonpath(data, '$..detail.textExtra[*].hashtagName')
print(tag)

author_info_key = ['uid','nickname','followerCount','totalFavorited','enterpriseVerifyReason','customVerify']
author_info_value = jsonpath.jsonpath(data, '$..detail.authorInfo[uid,nickname,followerCount,totalFavorited,enterpriseVerifyReason,customVerify]')
author_info = dict(zip(author_info_key, author_info_value))
print(author_info)

In [ ]:
xxx = getVideoInfo('6713294851909209348')
print(xxx)
data_en = re.findall(r'<script id="RENDER_DATA" type="application/json">(.*?)</script>', xxx)[0]
data_all = requests.utils.unquote(data_en)
data_json = json.loads(data_all)
data_json
jsonpath.jsonpath(data_json, '$..detail.textExtra[*].hashtagName')

In [ ]:
parseVideoInfo(getVideoInfo('6713294851909209348'))

In [ ]:
getVideoList_serach('冀中笙管乐军卢村音乐会')

In [ ]:
for keyword in keyword_df.iloc[267:, 4]:
    print(keyword)

In [ ]:
keyword_df[keyword_df['title'].str.contains('花儿')]['title']

In [10]:
keyword_videoList_dict = {}
error_keyword = []
for keyword in keyword_df.iloc[1481:, 4]:
    try:
        browser, flag = getVideoList_serach(keyword)
        result = {keyword: parseVideoList(browser=browser, flag=flag)}
        with open('./video_id_json/{}.json'.format(keyword), 'w', encoding='utf-8') as f:
            json.dump(result, f, ensure_ascii=False)
        # keyword_videoList_dict[keyword] = result
    except:
        # error_keyword.append(keyword)
        result = {keyword: [] }
        with open('./video_id_json/{}.json'.format(keyword), 'w', encoding='utf-8') as f:
            json.dump(result, f, ensure_ascii=False)
        # keyword_videoList_dict[keyword] = result
# pd.DataFrame(error_keyword,columns='keyword').to_csv('./error_keyword.csv', index=False)

Going on
成功加载cookies
Going on
Going on
成功加载cookies
Going on
Going on
成功加载cookies
Going on
成功获取图片url
成功获取图片url
成功获取图片url
Going on
成功加载cookies
Going on
成功获取图片url
成功获取图片url
Going on
成功加载cookies
Going on
Going on
成功加载cookies
Going on
成功获取图片url
成功获取图片url
成功获取图片url
成功获取图片url
成功获取图片url
成功获取图片url
成功获取图片url
成功获取图片url
成功获取图片url
成功获取图片url
Going on
成功加载cookies
Going on
成功获取图片url
Going on
成功加载cookies
Going on
成功获取图片url
成功获取图片url
成功获取图片url
成功获取图片url
Going on
成功加载cookies
Going on
成功获取图片url
成功获取图片url
Going on
成功加载cookies
Going on
成功获取图片url
成功获取图片url
成功获取图片url
Going on
成功加载cookies
Going on
成功获取图片url
Going on
成功加载cookies
Going on
成功获取图片url
成功获取图片url
成功获取图片url
成功获取图片url
成功获取图片url
成功获取图片url
成功获取图片url
成功获取图片url
成功获取图片url
成功获取图片url
Going on
成功加载cookies
Going on
成功获取图片url
成功获取图片url
成功获取图片url
成功获取图片url
成功获取图片url
Going on
成功加载cookies
Going on
成功获取图片url
成功获取图片url
成功获取图片url
成功获取图片url
成功获取图片url
Going on
成功加载cookies
Going on
成功获取图片url
成功获取图片url
成功获取图片url
成功获取图片url
Going on
成功加载cookies
Going on
成功获取图片url
成功获取图片url


In [2]:
import json
import requests
import urllib


def get_token():
    username = "test"
    password = "test"
    url = "https://api-v2.douyin.wtf/token"
    payload='grant_type=&username={}&password={}&scope=&client_id=&client_secret='.format(username, password)
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    response = requests.request("POST", url, headers=headers, data=payload)
    return response

def get_video_info_API(response, video_id):
    token_info = json.loads(response.text)
    url = "https://api-v2.douyin.wtf//douyin_video_data/?video_id={}&cursor=0&count=20".format(video_id)
    headers = {'Content-Type': 'accept: application/json',
    'authorization':'Bearer {}'.format(token_info['access_token'])}
    response = requests.request("GET", url, headers=headers)
    info = json.loads(response.text)
    return info

def get_video_comment_API(response, video_id):  
    token_info = json.loads(response.text)
    url = "https://api-v2.douyin.wtf/douyin_video_comments/?video_id={}&cursor=0&count=20".format(video_id)
    headers = {'Content-Type': 'accept: application/json',
    'authorization':'Bearer {}'.format(token_info['access_token'])}
    response = requests.request("GET", url, headers=headers)
    comment = json.loads(response.text)
    return comment
# with open('./DOUYIN_APItest.json', 'w', encoding='utf-8') as f:
#     json.dump(x,f,ensure_ascii=False)

API V1 (12.25 update):

In [7]:
import json
import requests
import urllib

def get_video_info_API_V1(video_id):
    url = "https://api.douyin.wtf/douyin_video_data/?video_id={}".format(video_id)
    headers = {'Content-Type': 'accept: application/json'}
    response = requests.request("GET", url, headers=headers)
    info = json.loads(response.text)
    return info

In [26]:
tttt = get_video_info_API_V1('7165126495612964103')
print(type(tttt))
tttt
awe_list = jsonpath.jsonpath(tttt, '$.aweme_list[*]')[0]
awe_list

<class 'dict'>


{'admire_auth': {'admire_button': 1,
  'is_admire': 0,
  'is_show_admire_button': 0,
  'is_show_admire_tab': 0},
 'anchors': None,
 'author': {'avatar_thumb': {'height': 720,
   'uri': '100x100/aweme-avatar/tos-cn-i-0813_dfa26cecc69346928a41a3de79787aa1',
   'url_list': ['https://p26.douyinpic.com/aweme/100x100/aweme-avatar/tos-cn-i-0813_dfa26cecc69346928a41a3de79787aa1.webp?from=116350172',
    'https://p3.douyinpic.com/aweme/100x100/aweme-avatar/tos-cn-i-0813_dfa26cecc69346928a41a3de79787aa1.webp?from=116350172',
    'https://p11.douyinpic.com/aweme/100x100/aweme-avatar/tos-cn-i-0813_dfa26cecc69346928a41a3de79787aa1.webp?from=116350172',
    'https://p26.douyinpic.com/aweme/100x100/aweme-avatar/tos-cn-i-0813_dfa26cecc69346928a41a3de79787aa1.jpeg?from=116350172'],
   'width': 720},
  'cf_list': None,
  'close_friend_type': 0,
  'contacts_status': 1,
  'contrail_list': None,
  'cover_url': [{'height': 720,
    'uri': 'c8510002be9a3a61aad2',
    'url_list': ['https://p9-sign.douyinpic.c

## USE API

In [28]:
import traceback
count = 0
path = './video_id_json/'
flag = False
for file_name in os.listdir(path):
    keyword = file_name.split('.')[0]
    if keyword == '蒙古族四胡音乐':
        flag = True
    if flag:
        print(keyword)
        with open('./video_id_json/{}'.format(file_name), 'r', encoding='utf-8') as f:
            video_id_json = json.load(f)
        for video_id in list(video_id_json[keyword]):
            if count>800:
                time.sleep(100)
                count = 0
            time.sleep(2.5)
            try:
                video_info = get_video_info_API_V1(video_id)
                awe_list = jsonpath.jsonpath(video_info, '$.aweme_list[*]')[0]
                if awe_list != None:
                    with open('./video_info_json/info_{}_{}.json'.format(keyword, video_id), 'w', encoding='utf-8') as f:
                        json.dump(awe_list, f, ensure_ascii=False)
                    print('{} API有效 加载成功'.format(video_id))
                else:
                    print('{} API失效 加载失败'.format(video_id))
            except Exception as e:
                print('https://www.douyin.com/video/{}  加载失败'.format(video_id))
                traceback.print_exc()
            count += 1

蒙古族四胡音乐
7172391906930855198 API有效 加载成功
7130507068829748517 API有效 加载成功
7062891448953294095 API有效 加载成功
7058466379871145251 API有效 加载成功
7098151548932394252 API有效 加载成功
7149030087218842920 API有效 加载成功
7174007964284488973 API有效 加载成功
7051150598837767439 API有效 加载成功
7056363493330554146 API有效 加载成功
7130126320847080741 API有效 加载成功
7051002391637806376 API有效 加载成功
7023653978722405672 API有效 加载成功
6922769499187989760 API有效 加载成功
7050023100762574121 API有效 加载成功
7134171329233734949 API有效 加载成功
7156100960450841863 API有效 加载成功
蒙古族安代舞
6871545127387614471 API有效 加载成功
7169070117782048013 API有效 加载成功
7057823621221289250 API有效 加载成功
6991816456631651584 API有效 加载成功
6813281913512529167 API有效 加载成功
7039325705657781534 API有效 加载成功
6806999033555537167 API有效 加载成功
6982022855110216967 API有效 加载成功
7123705749632535816 API有效 加载成功
7163880754118774047 API有效 加载成功
7131522463283924258 API有效 加载成功
6819097160898874628 API有效 加载成功
6826167099405241603 API有效 加载成功
6993934314236923144 API有效 加载成功
7145466891140664615 API有效 加载成功
7013746077388098846 API有

Traceback (most recent call last):
  File "C:\Users\Unknown\AppData\Local\Temp\ipykernel_36968\2243042018.py", line 20, in <module>
    awe_list = jsonpath.jsonpath(video_info, '$.aweme_list[*]')[0]
TypeError: 'bool' object is not subscriptable


7160546757011606817 API有效 加载成功
7049753877628964134 API有效 加载成功
7158024690240425229 API有效 加载成功
蒙古族民歌 和硕特民歌
7178466810172329249 API有效 加载成功
7005786287600979240 API有效 加载成功
6896844662594751758 API有效 加载成功
7099249343374331140 API有效 加载成功
6830623907234532612 API有效 加载成功
7051899616383569156 API有效 加载成功
6696803192962845963 API有效 加载成功
7155395200318311713 API有效 加载成功
7098691370503687462 API有效 加载成功
7043391715239103751 API有效 加载成功
7138357427128290575 API有效 加载成功
6938776240610381093 API有效 加载成功
7103354795460660491 API有效 加载成功
7128560634819546406 API有效 加载成功
7139106353770925351 API有效 加载成功
7155672621957352744 API有效 加载成功
6830311048709459213 API有效 加载成功
6929558547751570688 API有效 加载成功
7136568268462099716 API有效 加载成功
6985814036529122596 API有效 加载成功
6887115904824511755 API有效 加载成功
6964959916209491234 API有效 加载成功
6799789004754554120 API有效 加载成功
7154241836435983654 API有效 加载成功
6987023035802979595 API有效 加载成功
蒙古族民歌 新疆温泉县蒙古族短调民歌
6746605569785810190 API有效 加载成功
7156646716962278693 API有效 加载成功
7157682456622157093 API有效 加载成功
68492641

Traceback (most recent call last):
  File "C:\Users\Unknown\AppData\Local\Temp\ipykernel_36968\2243042018.py", line 20, in <module>
    awe_list = jsonpath.jsonpath(video_info, '$.aweme_list[*]')[0]
TypeError: 'bool' object is not subscriptable


7083448559063862539 API有效 加载成功
6701941035775331596 API有效 加载成功
6683447033455578371 API有效 加载成功
7084545357396380959 API有效 加载成功
7140586119489506594 API有效 加载成功
6684865887985470727 API有效 加载成功
7106697281507757350 API有效 加载成功
6862257544245579011 API有效 加载成功
7031498524118289671 API有效 加载成功
6652277788256587011 API有效 加载成功
6714495254059404557 API有效 加载成功
藏族唐卡 甘南藏族唐卡
6972490399017340167 API有效 加载成功
7101578034863508772 API有效 加载成功
6823179456119704847 API有效 加载成功
7150231190887353607 API有效 加载成功
6846970023228099840 API有效 加载成功
7134996813882330376 API有效 加载成功
7051506127497956621 API有效 加载成功
7110150363004636455 API有效 加载成功
7108605600115789095 API有效 加载成功
6967149675191225631 API有效 加载成功
6967205365926661415 API有效 加载成功
7108284313707367692 API有效 加载成功
7086753977206934791 API有效 加载成功
7153964801499188521 API有效 加载成功
7111291475983453473 API有效 加载成功
7108640387685829891 API有效 加载成功
藏族唐卡 藏娘唐卡
7040421464226450703 API有效 加载成功
7099654689951649061 API有效 加载成功
7176936068900490508 API有效 加载成功
6674533856701385996 API有效 加载成功
69724176253935157

...

7080120978725932299 API有效 加载成功
7089727631481998599 API有效 加载成功
7112317944729734436 API有效 加载成功
7085274684194884901 API有效 加载成功
7090008178372496680 API有效 加载成功
7113403963302481186 API有效 加载成功
7150735427060714783 API有效 加载成功
7060331556308618530 API有效 加载成功
6957539109548215566 API有效 加载成功
7086992876550376744 API有效 加载成功
鼓舞 宜川胸鼓
6946536076299308322 API有效 加载成功
7104476421795433739 API有效 加载成功
7012010845236448545 API有效 加载成功
6907562796939070731 API有效 加载成功
6961637146239323395 API有效 加载成功
7063390320842509582 API有效 加载成功
7144273330994957581 API有效 加载成功
6946888513321176356 API有效 加载成功
6961925565959998753 API有效 加载成功
6948265311413996841 API有效 加载成功
7124337370069011727 API有效 加载成功
7058982224057666846 API有效 加载成功
7054543509839170830 API有效 加载成功
6945999136294472995 API有效 加载成功
6954314717791898917 API有效 加载成功
7054897071240842496 API有效 加载成功
鼓舞 平定武迓鼓
鼓舞 彝族花鼓舞
7175102605654330683 API有效 加载成功
7129494230174665997 API有效 加载成功
7142016374016068872 API有效 加载成功
6992739868166065443 API有效 加载成功
7097605250630094091 API有效 加载成功
6972754105840

Traceback (most recent call last):
  File "C:\Users\Unknown\AppData\Local\Temp\ipykernel_36968\2243042018.py", line 20, in <module>
    awe_list = jsonpath.jsonpath(video_info, '$.aweme_list[*]')[0]
TypeError: 'bool' object is not subscriptable


7135335454722772265 API有效 加载成功
6743444361784184077 API有效 加载成功
7128763847233621255 API有效 加载成功
6980221393120103714 API有效 加载成功
7140193151783882022 API有效 加载成功
6983223439100218662 API有效 加载成功
7017029701235428619 API有效 加载成功
6982175184237153573 API有效 加载成功
7041421009269935395 API有效 加载成功
6915389743085686016 API有效 加载成功
https://www.douyin.com/video/7163321226604154148  加载失败


Traceback (most recent call last):
  File "C:\Users\Unknown\AppData\Local\Temp\ipykernel_36968\2243042018.py", line 20, in <module>
    awe_list = jsonpath.jsonpath(video_info, '$.aweme_list[*]')[0]
TypeError: 'bool' object is not subscriptable


7060073965166824741 API有效 加载成功
6982057657184161028 API有效 加载成功
7016521402685623567 API有效 加载成功
鼓舞 武山旋鼓舞
7135797005636390185 API有效 加载成功
7129336322925350180 API有效 加载成功
7042121181012446477 API有效 加载成功
7112419261246672142 API有效 加载成功
6973502683441401123 API有效 加载成功
7160987050035006761 API有效 加载成功
6919033696297258253 API有效 加载成功
7052605943531572516 API有效 加载成功
6973497417492614413 API有效 加载成功
7015446243010809101 API有效 加载成功
7100839246826720546 API有效 加载成功
7104473239631498530 API有效 加载成功
7113047720985627938 API有效 加载成功
6959419281612360975 API有效 加载成功
7145469626619677960 API有效 加载成功
7112814277580426526 API有效 加载成功
鼓舞 稷山高台花鼓
6813097559695936775 API有效 加载成功
7121551830689123595 API有效 加载成功
7148286484498500903 API有效 加载成功
7144598869353975079 API有效 加载成功
6939372098964000007 API有效 加载成功
7088194756823616781 API有效 加载成功
6874094320249539855 API有效 加载成功
6869258892266851592 API有效 加载成功
6812954881217940749 API有效 加载成功
7121531580610284831 API有效 加载成功
7064908623239040256 API有效 加载成功
6941587891944459553 API有效 加载成功
7053275740330085665 

In [7]:
video_info = get_video_info_API_V1('7168064057830935808')
video_info

{'status': 'failed',
 'platform': 'douyin',
 'endpoint': '/douyin_video_data/',
 'message': '视频API数据获取失败/Failed to get video API data'}

In [8]:
import traceback
count = 0
path = './need_update/'
flag = False
for file_name in os.listdir(path):
    keyword = file_name[:-5].split('_')[1]
    video_id =  file_name[:-5].split('_')[2]
    print(keyword)
    if count>800:
        time.sleep(100)
        count = 0
    time.sleep(2.5)
    try:
        video_info = get_video_info_API_V1(video_id)
        awe_list = jsonpath.jsonpath(video_info, '$.aweme_list[*]')[0]
        if awe_list != None:
            with open('./video_info_json/info_{}_{}.json'.format(keyword, video_id), 'w', encoding='utf-8') as f:
                json.dump(awe_list, f, ensure_ascii=False)
            print('{} API有效 加载成功'.format(video_id))
        else:
            print('{} API失效 加载失败'.format(video_id))
    except Exception as e:
        print('https://www.douyin.com/video/{}  加载失败'.format(video_id))
        traceback.print_exc()
    count += 1

乌孜别克族埃希来 叶来
https://www.douyin.com/video/7168064057830935808  加载失败
佛教音乐 北武当庙寺庙音乐


Traceback (most recent call last):
  File "C:\Users\Unknown\AppData\Local\Temp\ipykernel_119720\3889017845.py", line 15, in <module>
    awe_list = jsonpath.jsonpath(video_info, '$.aweme_list[*]')[0]
TypeError: 'bool' object is not subscriptable


https://www.douyin.com/video/7127550802436164900  加载失败
佛教音乐 鱼山梵呗


Traceback (most recent call last):
  File "C:\Users\Unknown\AppData\Local\Temp\ipykernel_119720\3889017845.py", line 15, in <module>
    awe_list = jsonpath.jsonpath(video_info, '$.aweme_list[*]')[0]
TypeError: 'bool' object is not subscriptable


7020344592499281156 API有效 加载成功
佛教音乐 鱼山梵呗
https://www.douyin.com/video/7181259268828548413  加载失败
傩舞 乐安傩舞


...

https://www.douyin.com/video/7165021039620115712  加载失败
花鼓戏


Traceback (most recent call last):
  File "C:\Users\Unknown\AppData\Local\Temp\ipykernel_119720\3889017845.py", line 15, in <module>
    awe_list = jsonpath.jsonpath(video_info, '$.aweme_list[*]')[0]
TypeError: 'bool' object is not subscriptable


https://www.douyin.com/video/7181065588515622181  加载失败
荆州花鼓戏


Traceback (most recent call last):
  File "C:\Users\Unknown\AppData\Local\Temp\ipykernel_119720\3889017845.py", line 15, in <module>
    awe_list = jsonpath.jsonpath(video_info, '$.aweme_list[*]')[0]
TypeError: 'bool' object is not subscriptable


6828252051923799307 API有效 加载成功
荆州花鼓戏
6956417921123896576 API有效 加载成功
荆州花鼓戏
7055222647264693540 API有效 加载成功
荆州花鼓戏
7057514313531116840 API有效 加载成功
荆河戏
6926097782130478348 API有效 加载成功
荆河戏
7054895705491524876 API有效 加载成功
荆河戏
7090935356450688297 API有效 加载成功
草编 哈萨克族芨芨草编织技艺
6755734362295110925 API有效 加载成功
草编 哈萨克族芨芨草编织技艺
6837045390022315277 API有效 加载成功
草编 哈萨克族芨芨草编织技艺
7027462759369477410 API有效 加载成功
草编 哈萨克族芨芨草编织技艺
7117182093519523085 API有效 加载成功
草编 哈萨克族芨芨草编织技艺
7127227725105859847 API有效 加载成功
草编 哈萨克族芨芨草编织技艺
7131969570767719715 API有效 加载成功
草编 哈萨克族芨芨草编织技艺
7168841512023870755 API有效 加载成功
草编 徐行草编
6877740237062278407 API有效 加载成功
草编 徐行草编
7025152330022522112 API有效 加载成功
草编 徐行草编
7178912599927475513 API有效 加载成功
草编 沐川草龙
6862687280201583875 API有效 加载成功
草编 沐川草龙
6868798640282995982 API有效 加载成功
草编 沐川草龙
6988768920199105806 API有效 加载成功
草编 沐川草龙
6993662686345170209 API有效 加载成功
草编 湖口草龙
6936436648272006440 API有效 加载成功
草编 湖口草龙
6949480628324683008 API有效 加载成功
草编 湖口草龙
6950117613179571491 API有效 加载成功
草编 湖口草龙
6968255214440680745 API有效 加载成功
草编

In [29]:
keyword_set = set()
for file_name in os.listdir('./video_info_json'):
    keyword = file_name[:-5].split('_')[1]
    keyword_set.add(keyword)
len(keyword_set)

1504

In [ ]:
import traceback
count = 0
response = get_token()
for ich_type in keyword_df['type'].unique():
    path = './' + ich_type
    title_list = os.listdir(path)
    print(ich_type, len(title_list))
    for keyword in title_list:
        time.sleep(5)
        with open(path + '/{}/{}.json'.format(keyword,keyword), 'r', encoding='utf-8') as f:
            json_dict = json.load(f)
        for key in json_dict.keys():
            video_list = json_dict[key]
            for video_id in video_list:
                if count>3000:
                    response = get_token()
                    count = 0
                time.sleep(2)
                try:
                    video_info = get_video_info_API(response, video_id)
                    with open(path + '/{}/{}.json'.format(keyword,video_id), 'w', encoding='utf-8') as f:
                        json.dump(video_info, f, ensure_ascii=False)
                except Exception as e:
                    print('{}, {}, https://www.douyin.com/video/{}'.format(ich_type,keyword,video_id))
                    traceback.print_exc()
                count += 1

是否有认证，是否是少数民族内容，还有是否是偏远地区的社区。 （作者的年龄，签名，看看还能不能搞来地区）

In [4]:
with open('./video_info_json/info_灯彩 北京灯彩_7058614524760329503.json', 'r', encoding='utf-8') as f:
    json_dict = json.load(f)
title = jsonpath.jsonpath(json_dict, '$.desc')[0]
title

['随着冬奥和春节的临近，前门大街装点一新，整条街在富含传统元素的灯彩映衬下，夜色阑珊，流光溢彩，一派祥和的喜庆气氛#北京#古街古巷 ']

In [6]:
id_l = os.listdir('./video_id_json/')

In [31]:
def parse_info_json(path, filename, queue):
    video_id = filename[:-5].split('_')[-1]
    keyword = filename[:-5].split('_')[-2]
    with open(path + filename, 'r', encoding='utf-8') as f:
        json_dict = json.load(f)
    try:      
        title = jsonpath.jsonpath(json_dict, '$..desc')[0]
        author = jsonpath.jsonpath(json_dict, '$..author.nickname')[0]
        author_signature = jsonpath.jsonpath(json_dict, '$..author.signature')[0]
        author_verify = jsonpath.jsonpath(json_dict, '$..author.enterprise_verify_reason')[0]
        author_like = jsonpath.jsonpath(json_dict, '$..author.total_favorited')[0]
        video_url = jsonpath.jsonpath(json_dict, '$..video.play_addr.url_list[0]')[0]
        # print(video_url)
        tag = jsonpath.jsonpath(json_dict, '$..text_extra[*].hashtag_name')
        # print(tag)
        title = jsonpath.jsonpath(json_dict, '$..desc')[0]
        # print(title)
        duration = jsonpath.jsonpath(json_dict, '$..duration')[0]
        # print(duration)
        num_comment = jsonpath.jsonpath(json_dict, '$..statistics.comment_count')[0]
        # print(num_comment)
        num_like = jsonpath.jsonpath(json_dict, '$..statistics.digg_count')[0]
        # print(num_like)
        num_share = jsonpath.jsonpath(json_dict, '$..statistics.share_count')[0]
        # print(num_share)
        queue.put({'ich_type':keyword_df[keyword_df['title']==keyword]['type'], 'ich_name':keyword, 'video_id':video_id,'video_url': video_url, 
        'video_author': author, 'author_verify': author_verify, 'author_signature': author_signature, 'author_like':author_like, 'video_title': title, 
        'video_tag':tag, 'video_duration':duration, 'num_com':num_comment, 'num_like':num_like, 'num_share':num_share})

    except Exception as e:
        print(filename)

In [32]:
""" coding:utf-8"""
#coding:utf-8
import time
import multiprocessing


if __name__ == '__main__': 
    queue = multiprocessing.Queue()
    jobs = []
    biglist = []
    path = './video_info_json/'
    for filename in os.listdir(path):
        p = multiprocessing.Process(target=parse_info_json, args=(path, filename, queue))
        jobs.append(p)
        p.start()
    
    for p in jobs:
        p.join()
    
    results = [queue.get() for j in jobs]
    print(results)


In [39]:
import traceback
big_list = []
path = './video_info_json/'
count=1
for filename in os.listdir(path):
    video_id = filename[:-5].split('_')[-1]
    keyword = filename[:-5].split('_')[-2]
    with open(path + filename, 'r', encoding='utf-8') as f:
        json_dict = json.load(f)
    try:      
        size = jsonpath.jsonpath(json_dict, '$..video.bit_rate[0].play_addr.data_size')[0]
        if size < 100000000:
            title = jsonpath.jsonpath(json_dict, '$..desc')[0]
            author = jsonpath.jsonpath(json_dict, '$..author.nickname')[0]
            author_signature = jsonpath.jsonpath(json_dict, '$..author.signature')[0]
            author_verify = jsonpath.jsonpath(json_dict, '$..author.enterprise_verify_reason')[0]
            author_like = jsonpath.jsonpath(json_dict, '$..author.total_favorited')[0]
            video_url = jsonpath.jsonpath(json_dict, '$..video.bit_rate[0].play_addr.url_list[2]')[0]
            # print(video_url)
            tag = jsonpath.jsonpath(json_dict, '$..text_extra[*].hashtag_name')
            # print(tag)
            title = jsonpath.jsonpath(json_dict, '$..desc')[0]
            # print(title)
            duration = jsonpath.jsonpath(json_dict, '$..video.duration')[0]
            duration = float(duration/1000)
            # print(duration)
            num_comment = jsonpath.jsonpath(json_dict, '$..statistics.comment_count')[0]
            # print(num_comment)
            num_like = jsonpath.jsonpath(json_dict, '$..statistics.digg_count')[0]
            # print(num_like)
            num_share = jsonpath.jsonpath(json_dict, '$..statistics.share_count')[0]
            # print(num_share)
            big_list.append({'ich_type':keyword_df[keyword_df['title']==keyword]['type'], 'ich_name':keyword, 'video_id':video_id,'video_url': video_url, 
            'video_author': author, 'author_verify': author_verify, 'author_signature': author_signature, 'author_like':author_like, 'video_title': title, 
            'video_tag':tag, 'video_duration':duration, 'num_com':num_comment, 'num_like':num_like, 'num_share':num_share, 'size':size})
            print(count)
            count+=1
        else:
            print('{} {} too large, drop it'.format(filename, count))
            count+=1
    except Exception as e:
        # print(filename)
        print('{} {} false'.format(filename, count))
        count+=1
        traceback.print_exc()

1
2
3
4
5
6
7
8
9
10
11
12
info_一勾勾_7047809336822811919.json 13 too large, drop it
14
15
16
17
18
19
20
21
22
23
24
info_三弦书 南阳三弦书_7041415998066412808.json 25 too large, drop it
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
info_三弦书 沁州三弦书_6862859883599957256.json 42 too large, drop it
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
info_三棒鼓_6761678841594350856.json 58 too large, drop it
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
info_三跳 湖州三跳_7033779819502374174.json 83 too large, drop it
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
info_上党二簧_7126326835658575140.json 100 too large, drop it
101
102
103
104
105
info_上党八音会_6934277787406683400.json 106 too large, drop it
info_上党八音会_6934902539477470471.json 107 too large, drop it
info_上党八音会_6986593338312920332.json 108 too large, drop it
109
110
111
112
113
114
115
116
117
info_上党八音会_7105950718070099212.json 118 too large, drop it
119
info_上党八音会_7148635472770632973.json 120 too large, drop it
121
info_上党落子_6920

Traceback (most recent call last):
  File "C:\Users\Unknown\AppData\Local\Temp\ipykernel_128652\1628689869.py", line 11, in <module>
    size = jsonpath.jsonpath(json_dict, '$..video.bit_rate[0].play_addr.data_size')[0]
TypeError: 'bool' object is not subscriptable
Traceback (most recent call last):
  File "C:\Users\Unknown\AppData\Local\Temp\ipykernel_128652\1628689869.py", line 11, in <module>
    size = jsonpath.jsonpath(json_dict, '$..video.bit_rate[0].play_addr.data_size')[0]
TypeError: 'bool' object is not subscriptable
Traceback (most recent call last):
  File "C:\Users\Unknown\AppData\Local\Temp\ipykernel_128652\1628689869.py", line 11, in <module>
    size = jsonpath.jsonpath(json_dict, '$..video.bit_rate[0].play_addr.data_size')[0]
TypeError: 'bool' object is not subscriptable
Traceback (most recent call last):
  File "C:\Users\Unknown\AppData\Local\Temp\ipykernel_128652\1628689869.py", line 11, in <module>
    size = jsonpath.jsonpath(json_dict, '$..video.bit_rate[0].play_ad

660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
info_京西太平鼓 怪村太平鼓_7071038870787312935.json 742 too large, drop it
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
info_京韵大鼓_7006159621253516580.json 781 too large, drop it
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
info_传统插花_7039943649202933000.json 840 false
info_传统插花_7075674022314315038.json 841 false
info_传统插花_7085577887696309544.json 842 false
info_传统插花_7087221587367316766.

Traceback (most recent call last):
  File "C:\Users\Unknown\AppData\Local\Temp\ipykernel_128652\1628689869.py", line 11, in <module>
    size = jsonpath.jsonpath(json_dict, '$..video.bit_rate[0].play_addr.data_size')[0]
TypeError: 'bool' object is not subscriptable
Traceback (most recent call last):
  File "C:\Users\Unknown\AppData\Local\Temp\ipykernel_128652\1628689869.py", line 11, in <module>
    size = jsonpath.jsonpath(json_dict, '$..video.bit_rate[0].play_addr.data_size')[0]
TypeError: 'bool' object is not subscriptable
Traceback (most recent call last):
  File "C:\Users\Unknown\AppData\Local\Temp\ipykernel_128652\1628689869.py", line 11, in <module>
    size = jsonpath.jsonpath(json_dict, '$..video.bit_rate[0].play_addr.data_size')[0]
TypeError: 'bool' object is not subscriptable
Traceback (most recent call last):
  File "C:\Users\Unknown\AppData\Local\Temp\ipykernel_128652\1628689869.py", line 11, in <module>
    size = jsonpath.jsonpath(json_dict, '$..video.bit_rate[0].play_ad

853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886
887
888
889
890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
info_佛教音乐 塔尔寺花架音乐_7004278454321564931.json 955 too large, drop it
956
957
958
959
960
961
962
963
964
965
966
info_佛教音乐 塔尔寺花架音乐_7120355305472445709.json 967 too large, drop it
968
969
970
971
972
973
974
975
info_佛教音乐 大相国寺梵乐_6877122836989889806.json 976 too large, drop it
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993
994
995
996
997
998
999
1000
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
info_佛教音乐 楞严寺寺庙音乐_6999915371083959565.json 1022 too large, drop it
info_佛教音乐 楞严寺寺庙音乐_7022537898406776079.json 1023 to

...

24411
24412
24413
24414
24415
24416
24417
24418
24419
24420
24421
24422
24423
24424
24425
24426
24427
24428
24429
24430
24431
24432
24433
24434
24435
24436
24437
24438
24439
24440
24441
24442
24443
24444
24445
24446
24447
24448
24449
24450
24451
24452
24453
24454
24455
24456
24457
24458
24459
24460
24461
24462
24463
24464
24465
24466
24467
24468
24469
24470
24471
24472
24473
24474
24475
24476
24477
24478
24479
24480
24481
24482
24483
24484
24485
info_锣鼓杂戏_6827142775729851652.json 24486 too large, drop it
24487
24488
24489
24490
24491
24492
24493
24494
24495
24496
24497
24498
24499
info_锣鼓杂戏_7160424235586915614.json 24500 too large, drop it
24501
24502
24503
info_锣鼓艺术 中州大鼓_6992511602348002573.json 24504 too large, drop it
24505
24506
info_锣鼓艺术 中州大鼓_7050451096325721379.json 24507 too large, drop it
24508
info_锣鼓艺术 中州大鼓_7054876651447913769.json 24509 too large, drop it
24510
24511
24512
24513
24514
24515
info_锣鼓艺术 中州大鼓_7154719334068243720.json 24516 too large, drop it
24517
24518
24519
in

Traceback (most recent call last):
  File "C:\Users\Unknown\AppData\Local\Temp\ipykernel_128652\1628689869.py", line 11, in <module>
    size = jsonpath.jsonpath(json_dict, '$..video.bit_rate[0].play_addr.data_size')[0]
TypeError: 'bool' object is not subscriptable
Traceback (most recent call last):
  File "C:\Users\Unknown\AppData\Local\Temp\ipykernel_128652\1628689869.py", line 11, in <module>
    size = jsonpath.jsonpath(json_dict, '$..video.bit_rate[0].play_addr.data_size')[0]
TypeError: 'bool' object is not subscriptable


25406
25407
25408
25409
25410
25411
25412
info_青海平弦_7004790716333427971.json 25413 too large, drop it
25414
25415
25416
25417
25418
25419
25420
25421
25422
25423
info_青海平弦_7169435227880803597.json 25424 too large, drop it
info_青海平弦_7169435648607177992.json 25425 too large, drop it
25426
25427
25428
25429
25430
25431
25432
25433
25434
25435
25436
25437
25438
25439
info_青海汉族民间小调_7163523958636055819.json 25440 too large, drop it
25441
25442
25443
25444
info_青海越弦_6827616950978497795.json 25445 too large, drop it
info_青海越弦_6872535427304230158.json 25446 too large, drop it
25447
info_青海越弦_7003460224929582376.json 25448 too large, drop it
25449
25450
25451
info_青海越弦_7117114339072478476.json 25452 too large, drop it
25453
25454
25455
25456
25457
info_青海越弦_7169436764233403684.json 25458 too large, drop it
25459
25460
25461
25462
25463
25464
25465
25466
25467
25468
25469
25470
25471
25472
25473
25474
25475
25476
25477
25478
25479
25480
25481
25482
25483
25484
25485
25486
25487
25488
25489
25490


Traceback (most recent call last):
  File "C:\Users\Unknown\AppData\Local\Temp\ipykernel_128652\1628689869.py", line 11, in <module>
    size = jsonpath.jsonpath(json_dict, '$..video.bit_rate[0].play_addr.data_size')[0]
TypeError: 'bool' object is not subscriptable


27511
27512
27513
27514
27515
27516
27517
27518
27519
27520
27521
27522
27523
27524
27525
27526
27527
27528
27529
27530
27531
27532
27533
27534
27535
27536
27537
27538
27539
27540
27541
27542
27543
27544
27545
27546
27547
27548
27549
27550
27551
27552
27553
27554
27555
27556
27557
27558
27559
27560
27561
27562
27563
27564
27565
27566
27567
27568
27569
27570
27571
27572
27573
27574
27575
27576
27577
27578
27579
27580
27581
27582
27583
27584
27585
27586
27587
27588
27589
27590
27591
27592
27593
27594
27595
27596
27597
27598
27599
27600
27601
27602
info_龙舞 铜梁龙舞_6999838359233383710.json 27603 too large, drop it
27604
info_龙舞 铜梁龙舞_7064466305113394468.json 27605 too large, drop it
27606
27607
27608
27609
27610
27611
27612
27613
27614
27615
27616
27617
27618
27619
27620
27621
27622
27623
27624
27625
27626
27627
27628
27629
27630
27631
27632
27633
27634
info_龙舞 香火龙_6907487442820009223.json 27635 too large, drop it
27636
27637
27638
27639
27640
27641
27642
27643
27644
27645
27646
27647
27648
27

In [ ]:
big_list = []
for ich_type in keyword_df['type'].unique():
    path = './' + ich_type
    title_list = os.listdir(path)
    print(ich_type, len(title_list))
    for keyword in title_list:
        time.sleep(5)
        video_files = os.listdir(path + '/{}'.format(keyword))
        for video_file in video_files:
            if os.path.isfile(path + '/{}/{}'.format(keyword,video_file)):
                if video_file[:-5].isdecimal():
                    with open(path + '/{}/{}'.format(keyword,video_file), 'r', encoding='utf-8') as f:
                        json_dict = json.load(f)
                    # video_url = jsonpath.jsonpath(json_dict, '$.aweme_list[*].video.play_addr.url_list[0]')[0]
                    # print(video_url)
                    # tag = jsonpath.jsonpath(json_dict, '$.aweme_list[*].text_extra[*].hashtag_name')
                    # # print(tag)
                    # title = jsonpath.jsonpath(json_dict, '$.aweme_list[*].desc')[0]
                    # print(title)
                    # duration = jsonpath.jsonpath(json_dict, '$.aweme_list[*].duration')[0]
                    # print(duration)
                    # num_comment = jsonpath.jsonpath(json_dict, '$.aweme_list[*].statistics.comment_count')[0]
                    # print(num_comment)
                    # num_like = jsonpath.jsonpath(json_dict, '$.aweme_list[*].statistics.digg_count')[0]
                    # print(num_like)
                    # num_share = jsonpath.jsonpath(json_dict, '$.aweme_list[*].statistics.share_count')[0]
                    # print(num_share)
                    video_url = jsonpath.jsonpath(json_dict, '$..video.play_addr.url_list[0]')[0]
                    print(video_url)
                    tag = jsonpath.jsonpath(json_dict, '$..text_extra[*].hashtag_name')
                    # print(tag)
                    title = jsonpath.jsonpath(json_dict, '$..desc')[0]
                    print(title)
                    duration = jsonpath.jsonpath(json_dict, '$..duration')[0]
                    print(duration)
                    num_comment = jsonpath.jsonpath(json_dict, '$..statistics.comment_count')[0]
                    print(num_comment)
                    num_like = jsonpath.jsonpath(json_dict, '$..statistics.digg_count')[0]
                    print(num_like)
                    num_share = jsonpath.jsonpath(json_dict, '$..statistics.share_count')[0]
                    print(num_share)
                    big_list.append({'ich_type':ich_type, 'ich_name':keyword, 'video_id':video_file[:-5].split('_')[2],'video_url': video_url, 'video_title': title, 'video_tag':tag, 'video_duration':duration, 'num_com':num_comment, 'num_like':num_like, 'num_share':num_share})
                    

In [5]:
# -- coding: utf-8 --**
with open('./video_info_json/info_一勾勾_6885629306484133135.json', 'r', encoding='utf-8') as f:
    json_dict = json.load(f)
# video_url = jsonpath.jsonpath(json_dict, '$..video.bit_rate[0].play_addr.url_list[2]')[0]
# print(video_url)
# tag = jsonpath.jsonpath(json_dict, '$..text_extra[*].hashtag_name')
# print(tag)
# title = jsonpath.jsonpath(json_dict, '$..desc')[0]
# print(title)
# duration = jsonpath.jsonpath(json_dict, '$..duration')[0]
# print(duration)
# num_comment = jsonpath.jsonpath(json_dict, '$..statistics.comment_count')[0]
# print(num_comment)
# num_like = jsonpath.jsonpath(json_dict, '$..statistics.digg_count')[0]
# print(num_like)
# num_share = jsonpath.jsonpath(json_dict, '$..statistics.share_count')[0]
# print(num_share)
author_age = jsonpath.jsonpath(json_dict, '$..author.user_age')[0]
print(author_age)
fps = jsonpath.jsonpath(json_dict, '$..bit_rate[*].gear_name')
print(fps)

-1
['adapt_lowest_720_1', 'adapt_540_1']


In [40]:
ich_video_info = pd.DataFrame(big_list)
ich_video_info

,ich_type,ich_name,video_id,video_url,video_author,author_verify,author_signature,author_like,video_title,video_tag,video_duration,num_com,num_like,num_share,size
0,"825 传统戏剧 Name: type, dtype: object",一勾勾,6866264054491663620,https://api.amemv.com/aweme/v1/play/?video_id=...,戏曲票友吴善家,,爱好戏曲，特好京剧！,7150,我们又排地方戏(一勾勾)墙头记！我还是扮演那受气老头！,False,152.195,15,29,1,14781146
1,"825 传统戏剧 Name: type, dtype: object",一勾勾,6885629306484133135,https://api.amemv.com/aweme/v1/play/?video_id=...,临邑县融媒体中心,临邑县融媒体中心官方抖音号,新闻前沿 百姓身边\n权威发布 热点资讯\n投稿加V:lyxrmtzx,347737,#非物质文化遗产 一勾勾，#临邑 的传统剧种。,"[非物质文化遗产, 临邑]",17.995,12,160,18,1918818
2,"825 传统戏剧 Name: type, dtype: object",一勾勾,6906839221604994317,https://api.amemv.com/aweme/v1/play/?video_id=...,泉正甜,,做快乐的人，越变越年轻！歌颂祖国，歌颂人民，歌颂生活，歌颂爱情，歌颂生命，关注人间疾苦，解说...,12930,糅合了吕剧豫剧秦腔河北梆子的临邑戏曲文化遗产——一勾勾挺好听。,False,73.834,0,7,0,16665420
3,"825 传统戏剧 Name: type, dtype: object",一勾勾,6950577893105880335,https://api.amemv.com/aweme/v1/play/?video_id=...,戏曲票友吴善家,,爱好戏曲，特好京剧！,7150,我们的地方戏，国家级非遗，《一勾勾》剧又排完一出戏…,False,65.063,9,34,2,6387185
4,"825 传统戏剧 Name: type, dtype: object",一勾勾,6975071420242021647,https://api.amemv.com/aweme/v1/play/?video_id=...,居士,,,340,国家非遗（一勾勾），复排“王汉喜借年”。,False,312.313,14,30,0,25153379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26979,"1063 曲艺 Name: type, dtype: object",龙舟说唱,7162006074617302309,https://api.amemv.com/aweme/v1/play/?video_id=...,粤风堂,店铺账号,弘扬优秀岭南文化\n汇聚众多非遗项目\n探寻新奇有趣故事\n感受名家大师魅力,328,国家级非遗龙舟说唱未来会消失吗？#传统文化 #弘扬和传承民族文化 #非遗传承 #民俗文化 #...,"[传统文化, 弘扬和传承民族文化, 非遗传承, 民俗文化, 传递正能量, 非遗抖起来, 传扬...",91.439,0,4,1,7890263
26980,"1063 曲艺 Name: type, dtype: object",龙舟说唱,7163939438458916135,https://api.amemv.com/aweme/v1/play/?video_id=...,海韵教育,,教育就是一棵树摇动一棵树，一朵云推动一朵云，一个灵魂唤醒一个灵魂！,9956,顺德龙舟说唱丨学好本领建中华 #文艺汇演 #演出现场 #活动现场 #顺德龙舟说唱,"[文艺汇演, 演出现场, 活动现场, 顺德龙舟说唱]",460.406,0,0,0,57012293
26981,"1063 曲艺 Name: type, dtype: object",龙舟说唱,7163939831742008614,https://api.amemv.com/aweme/v1/play/?video_id=...,海韵教育,,教育就是一棵树摇动一棵树，一朵云推动一朵云，一个灵魂唤醒一个灵魂！,9956,顺德龙舟说唱丨守法欢乐渡假期 #文艺汇演 #文化旅游 #演出现场 #振兴乡村 #顺德龙舟说唱,"[文艺汇演, 文化旅游, 演出现场, 振兴乡村, 顺德龙舟说唱]",128.366,0,0,0,13133944
26982,"1063 曲艺 Name: type, dtype: object",龙舟说唱,7163940216791780645,https://api.amemv.com/aweme/v1/play/?video_id=...,海韵教育,,教育就是一棵树摇动一棵树，一朵云推动一朵云，一个灵魂唤醒一个灵魂！,9956,顺德龙舟说唱丨战疫情报民安 #振兴乡村 #活动现场 #文艺汇演 #顺德龙舟说唱,"[振兴乡村, 活动现场, 文艺汇演, 顺德龙舟说唱]",226.134,0,0,0,26289182


In [41]:
result = pd.merge(ich_video_info,keyword_df.loc[:, ['type','title', 'province', 'content']], left_on='ich_name', right_on='title', how='left')
result
result['ich_type']=result['type']
result.drop(columns=['title', 'type'], inplace=True)
result

,ich_type,ich_name,video_id,video_url,video_author,author_verify,author_signature,author_like,video_title,video_tag,video_duration,num_com,num_like,num_share,size,province,content
0,传统戏剧,一勾勾,6866264054491663620,https://api.amemv.com/aweme/v1/play/?video_id=...,戏曲票友吴善家,,爱好戏曲，特好京剧！,7150,我们又排地方戏(一勾勾)墙头记！我还是扮演那受气老头！,False,152.195,15,29,1,14781146,山东省临邑县,“一勾勾”是由高唐一带的鼓子秧歌衍化而成的地方剧种。高唐鼓子秧歌是一种民间演唱艺术，演唱...
1,传统戏剧,一勾勾,6885629306484133135,https://api.amemv.com/aweme/v1/play/?video_id=...,临邑县融媒体中心,临邑县融媒体中心官方抖音号,新闻前沿 百姓身边\n权威发布 热点资讯\n投稿加V:lyxrmtzx,347737,#非物质文化遗产 一勾勾，#临邑 的传统剧种。,"[非物质文化遗产, 临邑]",17.995,12,160,18,1918818,山东省临邑县,“一勾勾”是由高唐一带的鼓子秧歌衍化而成的地方剧种。高唐鼓子秧歌是一种民间演唱艺术，演唱...
2,传统戏剧,一勾勾,6906839221604994317,https://api.amemv.com/aweme/v1/play/?video_id=...,泉正甜,,做快乐的人，越变越年轻！歌颂祖国，歌颂人民，歌颂生活，歌颂爱情，歌颂生命，关注人间疾苦，解说...,12930,糅合了吕剧豫剧秦腔河北梆子的临邑戏曲文化遗产——一勾勾挺好听。,False,73.834,0,7,0,16665420,山东省临邑县,“一勾勾”是由高唐一带的鼓子秧歌衍化而成的地方剧种。高唐鼓子秧歌是一种民间演唱艺术，演唱...
3,传统戏剧,一勾勾,6950577893105880335,https://api.amemv.com/aweme/v1/play/?video_id=...,戏曲票友吴善家,,爱好戏曲，特好京剧！,7150,我们的地方戏，国家级非遗，《一勾勾》剧又排完一出戏…,False,65.063,9,34,2,6387185,山东省临邑县,“一勾勾”是由高唐一带的鼓子秧歌衍化而成的地方剧种。高唐鼓子秧歌是一种民间演唱艺术，演唱...
4,传统戏剧,一勾勾,6975071420242021647,https://api.amemv.com/aweme/v1/play/?video_id=...,居士,,,340,国家非遗（一勾勾），复排“王汉喜借年”。,False,312.313,14,30,0,25153379,山东省临邑县,“一勾勾”是由高唐一带的鼓子秧歌衍化而成的地方剧种。高唐鼓子秧歌是一种民间演唱艺术，演唱...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26979,曲艺,龙舟说唱,7162006074617302309,https://api.amemv.com/aweme/v1/play/?video_id=...,粤风堂,店铺账号,弘扬优秀岭南文化\n汇聚众多非遗项目\n探寻新奇有趣故事\n感受名家大师魅力,328,国家级非遗龙舟说唱未来会消失吗？#传统文化 #弘扬和传承民族文化 #非遗传承 #民俗文化 #...,"[传统文化, 弘扬和传承民族文化, 非遗传承, 民俗文化, 传递正能量, 非遗抖起来, 传扬...",91.439,0,4,1,7890263,广东省佛山市顺德区,龙舟歌在民间又称“唱龙舟”或简称“龙舟”，是流行于广东珠江三角洲地区的一种曲艺形式，一般...
26980,曲艺,龙舟说唱,7163939438458916135,https://api.amemv.com/aweme/v1/play/?video_id=...,海韵教育,,教育就是一棵树摇动一棵树，一朵云推动一朵云，一个灵魂唤醒一个灵魂！,9956,顺德龙舟说唱丨学好本领建中华 #文艺汇演 #演出现场 #活动现场 #顺德龙舟说唱,"[文艺汇演, 演出现场, 活动现场, 顺德龙舟说唱]",460.406,0,0,0,57012293,广东省佛山市顺德区,龙舟歌在民间又称“唱龙舟”或简称“龙舟”，是流行于广东珠江三角洲地区的一种曲艺形式，一般...
26981,曲艺,龙舟说唱,7163939831742008614,https://api.amemv.com/aweme/v1/play/?video_id=...,海韵教育,,教育就是一棵树摇动一棵树，一朵云推动一朵云，一个灵魂唤醒一个灵魂！,9956,顺德龙舟说唱丨守法欢乐渡假期 #文艺汇演 #文化旅游 #演出现场 #振兴乡村 #顺德龙舟说唱,"[文艺汇演, 文化旅游, 演出现场, 振兴乡村, 顺德龙舟说唱]",128.366,0,0,0,13133944,广东省佛山市顺德区,龙舟歌在民间又称“唱龙舟”或简称“龙舟”，是流行于广东珠江三角洲地区的一种曲艺形式，一般...
26982,曲艺,龙舟说唱,7163940216791780645,https://api.amemv.com/aweme/v1/play/?video_id=...,海韵教育,,教育就是一棵树摇动一棵树，一朵云推动一朵云，一个灵魂唤醒一个灵魂！,9956,顺德龙舟说唱丨战疫情报民安 #振兴乡村 #活动现场 #文艺汇演 #顺德龙舟说唱,"[振兴乡村, 活动现场, 文艺汇演, 顺德龙舟说唱]",226.134,0,0,0,26289182,广东省佛山市顺德区,龙舟歌在民间又称“唱龙舟”或简称“龙舟”，是流行于广东珠江三角洲地区的一种曲艺形式，一般...


In [30]:
result.to_csv('./douyin_ich_video_info.csv', index=False)

# Save Video in async method

In [1]:
import requests, os, time, pandas as pd, jsonpath, json, random
import aiohttp, asyncio, nest_asyncio
nest_asyncio.apply()

In [2]:
result = pd.read_csv('./douyin_ich_video_info.csv')

### get some sample dataset

In [ ]:
result = pd.read_csv('./douyin_ich_video_info.csv')
print(len(result), result['size'].mean())
random_sample_result = result.sample(frac=0.65)
print(len(random_sample_result))
random_sample_result['size'].mean()
random_sample_result.to_csv('./douyin_ich_video_info_random_sample.csv', index=False)

read sample dataset

In [5]:
random_sample_result = pd.read_csv('./douyin_ich_video_info_random_sample.csv')


## use async way to download the ICH video

In [4]:
headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36",
    }

class DownloadVideo(object):
    def __init__(self):
        self.headers =  {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36",
    }

    async def download_video(self, video_url):
        retry_count = 0
        while retry_count < 5:
            ich_name = result[result['video_url']==video_url]['ich_name'].values[0]
            video_id = result[result['video_url']==video_url]['video_id'].values[0]
            try:
                async with aiohttp.ClientSession() as session:
                    async with session.get(video_url, headers=headers) as response:
                        content = await response.read()
                        with open('./video_content/{}_{}.mp4'.format(ich_name, video_id), 'wb') as f:
                            f.write(content)
                        print('downloaded')
                break
            except:
                print('{},{} download failed'.format(ich_name, video_id))
                print('It\'s url is: ',video_url)
                retry_count += 1

    def run(self, video_url_list):
        for video_url in video_url_list:
            tasks = asyncio.ensure_future(self.download_video(video_url))
        loop = asyncio.get_event_loop()
        loop.run_until_complete(tasks)


In [12]:
def main():
    simple_65_video_list = [int(x[:-4].split('_')[1]) for x in os.listdir('./video_content')]
    v_l = result.loc[~result['video_id'].isin(simple_65_video_list), 'video_url'].tolist()
    print(len(v_l))
    count = 0
    while count<len(v_l):
        video_url_list = v_l[count:count+12]
        count += 12
        dv_ich = DownloadVideo()
        dv_ich.run(video_url_list)
    video_url_list = v_l[count-12:]
    dv_ich = DownloadVideo()
    dv_ich.run(video_url_list)

if __name__ == '__main__':
    main()

9041
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downl

downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
downloaded
龙舞 鳌江划大龙,6834661548972166415 download failed
It's url is:  https://api.amemv.com/aweme/v1/play/?video_id=v0200fa50000brcpgflds13cflqeuddg&line=0&file_id=dd12c581df0b4397ae326ad53c1b102f&sign=eaa03d38ccf6684ed137aba1443c6a9a&is_play_url=1&source=PackSourceEnum_AWEME_DETAIL
downloaded
downloaded
downloaded
downloaded
downloaded


## USE SELFCODE

In [ ]:
import traceback

for ich_type in keyword_df['type'].unique():
    path = './' + ich_type
    title_list = os.listdir(path)
    print(ich_type, len(title_list))
    for keyword in title_list:
        time.sleep(5)
        with open(path + '/{}/{}.json'.format(keyword,keyword), 'r', encoding='utf-8') as f:
            json_dict = json.load(f)
        for key in json_dict.keys():
            video_list = json_dict[key]
            for video_id in video_list:
                time.sleep(2)
                try:
                    video_info = parseVideoInfo(getVideoInfo(video_id))
                    with open(path + '/{}/{}.json'.format(keyword,video_id), 'w', encoding='utf-8') as f:
                        json.dump(video_info, f, ensure_ascii=False)
                except Exception as e:
                    print('{}, {}, https://www.douyin.com/video/{}'.format(ich_type,keyword,video_id))
                    traceback.print_exc()
                    

## Save Video

In [ ]:
os.getcwd()

In [ ]:
import traceback
def savevideo(video_info_df:pd.DataFrame):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36",
    }
    names = video_info_df['ich_name'].unique()
    for name in names[49:]:
        temp_video_df = video_info_df[video_info_df['ich_name'] == name]
        for i in range(min(6, len(temp_video_df))):
            try:
                video_ich_type = temp_video_df.iloc[i]['ich_type']
                video_url = temp_video_df.iloc[i]['video_url']
                video_id = temp_video_df.iloc[i]['video_id']
                r = requests.get(video_url, headers=headers)
                with open('./video/{}_{}_{}.mp4'.format(video_ich_type, name, video_id), 'wb') as f:
                    f.write(r.content)
            except Exception as e:
                print('{}, {}, {}'.format(video_ich_type, name, video_url))
                traceback.print_exc()

In [ ]:
import pandas as pd

view_data = pd.read_csv('./douyin_ich_video_info_temp.csv')
view_data[]

In [ ]:
view_data[view_data['video_duration']<90000]['video_duration'].mean()

In [ ]:
useful_data = view_data[view_data['video_duration']<90000]

In [ ]:
print(useful_data['video_duration'].min(), useful_data['video_duration'].max())

In [ ]:
useful_data['video_duration'].std()

In [ ]:
0.086 /1.2

In [ ]:
xnums = [0.086,0.013,6.38,8.041,7.251,1.11,13.880,4.790,2.90,-79.250, 34.436,-2.3,11700,11653.755,-1, -2392.941,1670.863,-1.43,-37800,18042.848,-2.1,-15000,9802.836,-1.53]
cod = 1.2005644787195
for xnum in xnums:
    res = xnum /cod
    print(',',res)